In [100]:
import os
import openai
import pandas as pd
import pdfplumber
import json
import numpy as np

import config

In [18]:
from openpyxl import load_workbook

file_path = '../Downloads/all_studies_final.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active

# Convert to DataFrame
data = sheet.values
columns = next(data)
df_truth = pd.DataFrame(data, columns=columns)

In [2]:
#%env OPENAI_API_KEY= API_KEY

In [4]:
import openai
from openai import OpenAI
client = OpenAI()

In [82]:
def rename_columns(df):
    # Rename columns in df to match df_truth
    df = df.rename(columns={
        "Last Name of Main Author and Year": "References",
        "Full Population Sample Size": "Sample size",
        "Main Race": "Main race",
        "Intervention": "Interventions",
        "Percent of Study Population that is Female (%)": "Female (%)",
        "Mean HAMA Score": "Mean HAMA",
        "Mean Population Age (Year)": "Mean age (Year)",
        "Attrition Rate (%)": "Attrition rate (%)",
        "Full Sponsor Name": "Sponsor",
        "Follow-up Time (Weeks)": "Follow-up time (weeks)",
        "Diagnostic Criteria": "Diagnosis criteria"
    })
    return df

# Training

In [6]:
def get_file_names(directory):
    """Gets a list of file names in the specified directory.

    Args:
        directory (str): The path to the directory.

    Returns:
        list: A list of file names.
    """

    file_names = []
    for entry in os.scandir(directory):
        if entry.is_file():
            file_names.append(entry.name)
    return file_names

directory_path = "../Desktop/Training"
file_names = get_file_names(directory_path)
print(file_names)

['stein2017.pdf', 'bidzan2012.pdf', 'coric2010.pdf', 'stein2014.pdf', 'feltner2003.pdf', 'koponen2007.pdf', 'montgomery2008.pdf', 'pollock2008b.pdf', 'bandelow2010.pdf', 'mezhebovsky2013.pdf']


# Training Set - Text only

In [7]:
# Function to extract text and tables
def extract_pdf_text(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            # Extract text
            text += page.extract_text() + "\n"
    return text


def query_gpt4_text(text):
    structured_prompt = (
        f"Clinical Trial Report Analysis:\n\n"
        f"Extracted Text:\n{text}\n\n"
        f"This is a clinical trial report. For EACH intervention in the trial (including placebo), please extract the following characteristics and format the response as valid JSON using this exact example structure:\n\n"
        f"Example format:\n"
        f"{{\n"
        f'    "Last Name of Main Author and Year": "Doe et al., 2021",\n'
        f'    "Full Population Sample Size": "451",\n'
        f'    "Intervention": "Duloxetine: 50 mg/day",\n'
        f'    "Main Race": "White",\n'
        f'    "Percent of Study Population that is Female (%)": "61.5",\n'
        f'    "Mean HAMA Score": "24.5",\n'
        f'    "Mean Population Age (Year)": "43.2",\n'
        f'    "Attrition Rate (%)": "30.2",\n'
        f'    "Full Sponsor Name": "ABC Pharmaceuticals",\n'
        f'    "Follow-up Time (Weeks)": "10",\n'
        f'    "Diagnostic Criteria": "DSM-IV"\n'
        f"}}\n\n"
        f"'Full Population Sample Size' should refer to the TOTAL population enrolled in the study, across all interventions and groups, not just the population size for the specific intervention.\n"
        f"Make sure each JSON object follows this format exactly. For any missing or unavailable data, input 'NA'."
    )
    response = completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a data extraction assistant extracting data from clinical trial reports."},
                  {"role": "user", "content": structured_prompt}],
    )
    return response

## Stein 2017

In [ ]:
file_path = "../Desktop/Training/stein2017.pdf"

# Extract content from PDF
stein_2017_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_stein_2017 = query_gpt4_text(stein_2017_text)

In [12]:
print("Extracted Data:", structured_data_stein_2017.choices[0].message.content)

Extracted Data: ```json
{
    "Last Name of Main Author and Year": "Stein et al., 2017",
    "Full Population Sample Size": "412",
    "Intervention": "Agomelatine: 10 mg/day",
    "Main Race": "NA",
    "Percent of Study Population that is Female (%)": "67.7",
    "Mean HAMA Score": "28.6",
    "Mean Population Age (Year)": "43.9",
    "Attrition Rate (%)": "13.7",
    "Full Sponsor Name": "Institut de Recherches Internationales Servier (IRIS)",
    "Follow-up Time (Weeks)": "12",
    "Diagnostic Criteria": "DSM-IV-TR"
}
{
    "Last Name of Main Author and Year": "Stein et al., 2017",
    "Full Population Sample Size": "412",
    "Intervention": "Agomelatine: 25 mg/day",
    "Main Race": "NA",
    "Percent of Study Population that is Female (%)": "67.7",
    "Mean HAMA Score": "29.0",
    "Mean Population Age (Year)": "44.1",
    "Attrition Rate (%)": "9.4",
    "Full Sponsor Name": "Institut de Recherches Internationales Servier (IRIS)",
    "Follow-up Time (Weeks)": "12",
    "Diagn

In [16]:
clean_output(structured_data_stein_2017)

Error parsing JSON: Extra data: line 14 column 1 (char 504)


""


In [70]:
structured_data_stein_2017.choices[0].message.content

'```json\n{\n    "Last Name of Main Author and Year": "Stein et al., 2017",\n    "Full Population Sample Size": "412",\n    "Intervention": "Agomelatine: 10 mg/day",\n    "Main Race": "NA",\n    "Percent of Study Population that is Female (%)": "67.7",\n    "Mean HAMA Score": "28.6",\n    "Mean Population Age (Year)": "43.9",\n    "Attrition Rate (%)": "13.7",\n    "Full Sponsor Name": "Institut de Recherches Internationales Servier (IRIS)",\n    "Follow-up Time (Weeks)": "12",\n    "Diagnostic Criteria": "DSM-IV-TR"\n}\n{\n    "Last Name of Main Author and Year": "Stein et al., 2017",\n    "Full Population Sample Size": "412",\n    "Intervention": "Agomelatine: 25 mg/day",\n    "Main Race": "NA",\n    "Percent of Study Population that is Female (%)": "67.7",\n    "Mean HAMA Score": "29.0",\n    "Mean Population Age (Year)": "44.1",\n    "Attrition Rate (%)": "9.4",\n    "Full Sponsor Name": "Institut de Recherches Internationales Servier (IRIS)",\n    "Follow-up Time (Weeks)": "12",\n

In [80]:
# Step 1: Clean the text if wrapped in code block markers
text = structured_data_stein_2017.choices[0].message.content
text = text.replace("```json\n{\n", "```json\n[\n    {\n")
text = text.replace("\n}\n```", "\n    }\n]\n```")
text = text.replace("}\n{","},\n{")

if text.startswith("```json"):
    text = text[len("```json"):].strip()
if text.endswith("```"):
    text = text[:-len("```")].strip()

# Step 2: Parse the JSON list directly
try:
    data = json.loads(text)  # Parse the JSON list directly
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    data = []

# Step 3: Convert the list of dictionaries into a pandas DataFrame
stein_2017_df = pd.DataFrame(data)


In [88]:
stein_2017_df

,Last Name of Main Author and Year,Full Population Sample Size,Intervention,Main Race,Percent of Study Population that is Female (%),Mean HAMA Score,Mean Population Age (Year),Attrition Rate (%),Full Sponsor Name,Follow-up Time (Weeks),Diagnostic Criteria
0,"Stein et al., 2017",412,Agomelatine: 10 mg/day,NA,67.7,28.6,43.9,13.7,Institut de Recherches Internationales Servier...,12,DSM-IV-TR
1,"Stein et al., 2017",412,Agomelatine: 25 mg/day,NA,67.7,29.0,44.1,9.4,Institut de Recherches Internationales Servier...,12,DSM-IV-TR
2,"Stein et al., 2017",412,Placebo,NA,67.7,28.8,44.1,21.1,Institut de Recherches Internationales Servier...,12,DSM-IV-TR


In [89]:
stein_2017_df = rename_columns(stein_2017_df)

In [155]:
def preprocess_numerical_columns(df, numerical_cols):
    """
    Converts specified numerical columns to numeric types in a DataFrame.
    Keeps 'NA' as a string and does not convert it to NaN.
    """
    for col in numerical_cols:
        if col in df.columns:
            # Preserve 'NA' and convert the rest to numeric
            df[col] = df[col].apply(lambda x: x if str(x).strip().lower() == "na" else pd.to_numeric(x, errors="coerce"))
    return df

# Update the numerical columns list
numerical_cols = ["Sample size", "Female (%)", "Mean HAMA", "Mean age (Year)", "Attrition rate (%)", "Follow-up time (weeks)"]


In [91]:
df_truth = preprocess_numerical_columns(df_truth, numerical_cols)
stein_2017_df = preprocess_numerical_columns(stein_2017_df, numerical_cols)

In [92]:
def preprocess_interventions_and_diag(df):
    """
    Normalizes the 'Interventions' column in the DataFrame for consistent filtering.
    Replaces all dashes (e.g., en dash, em dash) with a standard hyphen.
    """
    df = df.copy()
    df["Interventions"] = (
        df["Interventions"]
        .str.strip()
        .str.lower()
        .str.replace("–", "-", regex=False)  # Replace en dash with hyphen
        .str.replace("—", "-", regex=False)  # Replace em dash with hyphen
        .str.replace("\u00a0", " ")  # Replace non-breaking space with regular space
    )
    df["Diagnosis criteria"] = (
    df["Diagnosis criteria"]
        .str.strip()
        .str.lower()
        .str.replace("–", "-", regex=False)  # Replace en dash with hyphen
        .str.replace("—", "-", regex=False)  # Replace em dash with hyphen
        .str.replace("\u00a0", " ")  # Replace non-breaking space with regular space
    )
    return df

In [93]:
# Normalize the 'Interventions' column in both DataFrames
df_truth = preprocess_interventions_and_diag(df_truth)
stein_2017_df = preprocess_interventions_and_diag(stein_2017_df)

In [164]:
# def testing(df_truth, df):
#     grouped_truth = df_truth.groupby("References")
#     grouped_pred = df.groupby("References")
    
#     results = []
    
#     for reference in grouped_truth.groups:
#         if reference in grouped_pred.groups:
#             # Get groups
#             truth_group = grouped_truth.get_group(reference)
#             pred_group = grouped_pred.get_group(reference)
    
#             # Normalize and find common interventions
#             truth_interventions = set(truth_group["Interventions"].tolist())
#             pred_interventions = set(pred_group["Interventions"].tolist())
#             common_interventions = truth_interventions & pred_interventions  # Only common interventions
    
#             for intervention in common_interventions:
#                 # Filter for the specific intervention
#                 truth_row = truth_group[truth_group["Interventions"] == intervention]
#                 pred_row = pred_group[pred_group["Interventions"] == intervention]
    
#                 if not truth_row.empty and not pred_row.empty:
#                     truth_row = truth_row.iloc[0]
#                     pred_row = pred_row.iloc[0]
    
#                     # Categorical columns with exact matching
#                     categorical_cols = ["Main race"]
#                     categorical_match = {
#                         col: truth_row[col].strip().lower() == pred_row[col].strip().lower()
#                         for col in categorical_cols
#                     }
    
#                     # Special handling for "Diagnosis criteria"
#                     diagnosis_criteria_truth = truth_row["Diagnosis criteria"].strip().lower()
#                     diagnosis_criteria_pred = pred_row["Diagnosis criteria"].strip().lower()
#                     categorical_match["Diagnosis criteria"] = diagnosis_criteria_truth in diagnosis_criteria_pred

#                     # Special handling for "Sponsor"
#                     sponsor_truth = truth_row["Sponsor"].strip().lower()
#                     sponsor_pred = pred_row["Sponsor"].strip().lower()
#                     categorical_match["Sponsor"] = sponsor_truth in sponsor_pred

#                     # Numerical columns
#                     numerical_cols = ["Sample size", "Female (%)", "Mean HAMA", "Mean age (Year)", "Attrition rate (%)", "Follow-up time (weeks)"]
#                     numerical_match = {
#                         col: np.isclose(truth_row[col], pred_row[col], atol=.5, equal_nan=True)
#                         for col in numerical_cols
#                     }
    
#                     # Collect results
#                     results.append({
#                         "References": reference,
#                         "Interventions": intervention,
#                         **categorical_match,
#                         **numerical_match,
#                     })
#     results_df = pd.DataFrame(results)
#     return results_df

In [172]:
def testing(df_truth, df):
    grouped_truth = df_truth.groupby("References")
    grouped_pred = df.groupby("References")
    
    results = []
    
    for reference in grouped_truth.groups:
        if reference in grouped_pred.groups:
            # Get groups
            truth_group = grouped_truth.get_group(reference)
            pred_group = grouped_pred.get_group(reference)
    
            # Normalize and find common interventions
            truth_interventions = set(truth_group["Interventions"].tolist())
            pred_interventions = set(pred_group["Interventions"].tolist())
            common_interventions = truth_interventions & pred_interventions  # Only common interventions
    
            for intervention in common_interventions:
                # Filter for the specific intervention
                truth_row = truth_group[truth_group["Interventions"] == intervention]
                pred_row = pred_group[pred_group["Interventions"] == intervention]
    
                if not truth_row.empty and not pred_row.empty:
                    truth_row = truth_row.iloc[0]
                    pred_row = pred_row.iloc[0]
    
                    # Categorical columns
                    categorical_cols = ["Main race", "Sponsor"]
                    categorical_match = {
                        col: truth_row[col].strip().lower() == pred_row[col].strip().lower()
                        for col in categorical_cols
                    }
    
                    # Special handling for "Diagnosis criteria"
                    diagnosis_criteria_truth = truth_row["Diagnosis criteria"].strip().lower()
                    diagnosis_criteria_pred = pred_row["Diagnosis criteria"].strip().lower()
                    categorical_match["Diagnosis criteria"] = diagnosis_criteria_truth in diagnosis_criteria_pred

                    # Numerical columns
                    numerical_cols = ["Sample size", "Female (%)", "Mean HAMA", "Mean age (Year)", "Attrition rate (%)", "Follow-up time (weeks)"]
                    numerical_match = {}
                    for col in numerical_cols:
                        truth_val = truth_row[col]
                        pred_val = pred_row[col]
                        truth_is_na = str(truth_val).strip().lower() == "na"
                        pred_is_na = str(pred_val).strip().lower() == "na"

                        # Handle different scenarios for NA
                        if truth_is_na and pred_is_na:
                            numerical_match[col] = True  # Both are NA, match is True
                        elif truth_is_na or pred_is_na:
                            numerical_match[col] = False  # Only one is NA, match is False
                        else:
                            # Perform numerical comparison
                            numerical_match[col] = np.isclose(
                                float(truth_val), float(pred_val), atol=0.5, equal_nan=True
                            )
    
                    # Collect results
                    results.append({
                        "References": reference,
                        "Interventions": intervention,
                        **categorical_match,
                        **numerical_match,
                    })
    results_df = pd.DataFrame(results)
    return results_df

In [119]:
stein_2017_df

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
0,"Stein et al., 2017",412,agomelatine: 10 mg/day,NA,67.7,28.6,43.9,13.7,Institut de Recherches Internationales Servier...,12,dsm-iv-tr
1,"Stein et al., 2017",412,agomelatine: 25 mg/day,NA,67.7,29.0,44.1,9.4,Institut de Recherches Internationales Servier...,12,dsm-iv-tr
2,"Stein et al., 2017",412,placebo,NA,67.7,28.8,44.1,21.1,Institut de Recherches Internationales Servier...,12,dsm-iv-tr


In [120]:
df_truth[df_truth["References"] == "Stein et al., 2017"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
20,"Stein et al., 2017",412,agomelatine: 10 mg/day,NA,67.9,28.6,43.6,13.7,Servier,12.0,dsm-iv
21,"Stein et al., 2017",412,agomelatine: 25 mg/day,NA,71.9,29.0,44.1,9.4,Servier,12.0,dsm-iv
22,"Stein et al., 2017",412,placebo,NA,63.4,28.8,44.1,21.1,Servier,12.0,dsm-iv


In [147]:
stein2017_results = testing(df_truth, stein_2017_df)

In [148]:
stein2017_results

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Stein et al., 2017",agomelatine: 10 mg/day,True,True,True,True,True,True,True,True,True
1,"Stein et al., 2017",placebo,True,True,True,True,False,True,True,True,True
2,"Stein et al., 2017",agomelatine: 25 mg/day,True,True,True,True,False,True,True,True,True


In [149]:
def ensure_all_truth_interventions_present(results_df, df_truth):
    """
    Ensures all interventions in `df_truth` are present in `results_df`, grouped by reference.
    If an intervention is missing, adds a row with False for all metrics.
    """
    # Normalize `Interventions` in `df_truth` to match `results_df`
    df_truth = preprocess_interventions_and_diag(df_truth)
    df_truth = df_truth[df_truth["References"].isin(results_df["References"])]

    # Group by reference in both dataframes
    grouped_truth = df_truth.groupby("References")
    grouped_results = results_df.groupby("References")

    # Initialize a list to store missing rows
    missing_rows = []

    # Iterate through each reference in `df_truth`
    for reference, truth_group in grouped_truth:
        # Get the interventions for the current reference in `df_truth`
        truth_interventions = set(truth_group["Interventions"].tolist())

        # Get the interventions for the same reference in `results_df`, if it exists
        if reference in grouped_results.groups:
            results_group = grouped_results.get_group(reference)
            results_interventions = set(results_group["Interventions"].tolist())
        else:
            results_interventions = set()

        # Find missing interventions for this reference
        missing_interventions = truth_interventions - results_interventions

        # Create rows for missing interventions
        for intervention in missing_interventions:
            missing_rows.append({
                "References": reference,
                "Interventions": intervention,
                **{col: False for col in results_df.columns if col not in ["References", "Interventions"]}
            })

    # Add missing rows to results_df
    if missing_rows:
        results_df = pd.concat([results_df, pd.DataFrame(missing_rows)], ignore_index=True)

    return results_df

In [127]:
stein2017_results_final = ensure_all_truth_interventions_present(stein2017_results,df_truth)

In [128]:
stein2017_results_final

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Stein et al., 2017",agomelatine: 10 mg/day,True,True,True,True,True,True,True,True,True
1,"Stein et al., 2017",placebo,True,True,True,True,False,True,True,True,True
2,"Stein et al., 2017",agomelatine: 25 mg/day,True,True,True,True,False,True,True,True,True


In [129]:
file_path = os.path.join("data_text", "stein2017data.csv")  
stein_2017_df.to_csv(file_path, index=False)  

file_path = os.path.join("results_text", "stein2017results.csv")  
stein2017_results_final.to_csv(file_path, index=False)  


# Bidzan 2012

In [130]:
file_path = "../Desktop/Training/bidzan2012.pdf"

# Extract content from PDF
bidzan_2012_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_bidzan_2012 = query_gpt4_text(bidzan_2012_text)

In [131]:
print("Extracted Data:", structured_data_bidzan_2012.choices[0].message.content)

Extracted Data: ```json
{
    "Last Name of Main Author and Year": "Bidzan et al., 2012",
    "Full Population Sample Size": "301",
    "Intervention": "Vortioxetine: 5 mg/day",
    "Main Race": "White",
    "Percent of Study Population that is Female (%)": "68.7",
    "Mean HAMA Score": "26.3",
    "Mean Population Age (Year)": "45.2",
    "Attrition Rate (%)": "NA",
    "Full Sponsor Name": "Takeda Pharmaceutical Company, Ltd.",
    "Follow-up Time (Weeks)": "8",
    "Diagnostic Criteria": "DSM-IV-TR"
}
{
    "Last Name of Main Author and Year": "Bidzan et al., 2012",
    "Full Population Sample Size": "301",
    "Intervention": "Placebo",
    "Main Race": "White",
    "Percent of Study Population that is Female (%)": "61.6",
    "Mean HAMA Score": "26.8",
    "Mean Population Age (Year)": "45.3",
    "Attrition Rate (%)": "NA",
    "Full Sponsor Name": "Takeda Pharmaceutical Company, Ltd.",
    "Follow-up Time (Weeks)": "8",
    "Diagnostic Criteria": "DSM-IV-TR"
}
```


In [135]:
#Let's make this a method
def json_parsing(output):
    text = output.choices[0].message.content
    text = text.replace("```json\n{\n", "```json\n[\n    {\n")
    text = text.replace("\n}\n```", "\n    }\n]\n```")
    text = text.replace("}\n{","},\n{")
    
    if text.startswith("```json"):
        text = text[len("```json"):].strip()
    if text.endswith("```"):
        text = text[:-len("```")].strip()
    
    # Step 2: Parse the JSON list directly
    try:
        data = json.loads(text)  # Parse the JSON list directly
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        data = []
    
    # Step 3: Convert the list of dictionaries into a pandas DataFrame
    return pd.DataFrame(data)


In [156]:
bidzan_2012_df = json_parsing(structured_data_bidzan_2012)

In [157]:
bidzan_2012_df

,Last Name of Main Author and Year,Full Population Sample Size,Intervention,Main Race,Percent of Study Population that is Female (%),Mean HAMA Score,Mean Population Age (Year),Attrition Rate (%),Full Sponsor Name,Follow-up Time (Weeks),Diagnostic Criteria
0,"Bidzan et al., 2012",301,Vortioxetine: 5 mg/day,White,68.7,26.3,45.2,NA,"Takeda Pharmaceutical Company, Ltd.",8,DSM-IV-TR
1,"Bidzan et al., 2012",301,Placebo,White,61.6,26.8,45.3,NA,"Takeda Pharmaceutical Company, Ltd.",8,DSM-IV-TR


In [145]:
df_truth[df_truth["References"] == "Bidzan et al., 2012"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
59,"Bidzan et al., 2012",301,vortioxetine: 5 mg/day,White,68.7,26.3,45.0,14.7,Takeda,8.0,dsm-iv
60,"Bidzan et al., 2012",301,placebo,White,61.6,26.8,45.3,16.6,Takeda,8.0,dsm-iv


In [158]:
bidzan_2012_df = rename_columns(bidzan_2012_df)
bidzan_2012_df = preprocess_interventions_and_diag(bidzan_2012_df)
bidzan_2012_df = preprocess_numerical_columns(bidzan_2012_df, numerical_cols)

In [159]:
bidzan_2012_df

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
0,"Bidzan et al., 2012",301,vortioxetine: 5 mg/day,White,68.7,26.3,45.2,NA,"Takeda Pharmaceutical Company, Ltd.",8,dsm-iv-tr
1,"Bidzan et al., 2012",301,placebo,White,61.6,26.8,45.3,NA,"Takeda Pharmaceutical Company, Ltd.",8,dsm-iv-tr


In [173]:
bidzan2012_results = testing(df_truth, bidzan_2012_df)

In [174]:
bidzan2012_results

,References,Interventions,Main race,Sponsor,Diagnosis criteria,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Bidzan et al., 2012",placebo,True,False,True,True,True,True,True,False,True
1,"Bidzan et al., 2012",vortioxetine: 5 mg/day,True,False,True,True,True,True,True,False,True


In [175]:
bidzan2012_results_final = ensure_all_truth_interventions_present(bidzan2012_results,df_truth)

In [176]:
bidzan2012_results_final

,References,Interventions,Main race,Sponsor,Diagnosis criteria,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Bidzan et al., 2012",placebo,True,False,True,True,True,True,True,False,True
1,"Bidzan et al., 2012",vortioxetine: 5 mg/day,True,False,True,True,True,True,True,False,True


In [177]:
file_path = os.path.join("data_text", "bidzan2012data.csv")  
bidzan_2012_df.to_csv(file_path, index=False)  

file_path = os.path.join("results_text", "bidzan2012results.csv")  
bidzan2012_results_final.to_csv(file_path, index=False)  


## Coric 2010

In [178]:
file_path = "../Desktop/Training/coric2010.pdf"

# Extract content from PDF
coric_2010_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_coric_2010 = query_gpt4_text(coric_2010_text)

In [180]:
print("Extracted Data:", structured_data_coric_2010.choices[0].message.content)

Extracted Data: ```json
[
    {
        "Last Name of Main Author and Year": "Coric et al., 2010",
        "Full Population Sample Size": "260",
        "Intervention": "Pexacerfont: 100 mg/day",
        "Main Race": "White",
        "Percent of Study Population that is Female (%)": "100",
        "Mean HAMA Score": "24.0",
        "Mean Population Age (Year)": "38.5",
        "Attrition Rate (%)": "5",
        "Full Sponsor Name": "Bristol-Myers Squibb Company",
        "Follow-up Time (Weeks)": "8",
        "Diagnostic Criteria": "DSM-IV-TR"
    },
    {
        "Last Name of Main Author and Year": "Coric et al., 2010",
        "Full Population Sample Size": "260",
        "Intervention": "Placebo",
        "Main Race": "White",
        "Percent of Study Population that is Female (%)": "100",
        "Mean HAMA Score": "24.7",
        "Mean Population Age (Year)": "39.3",
        "Attrition Rate (%)": "3",
        "Full Sponsor Name": "Bristol-Myers Squibb Company",
        "Follow-u

In [181]:
coric_2010_df = json_parsing(structured_data_coric_2010)

In [182]:
coric_2010_df

,Last Name of Main Author and Year,Full Population Sample Size,Intervention,Main Race,Percent of Study Population that is Female (%),Mean HAMA Score,Mean Population Age (Year),Attrition Rate (%),Full Sponsor Name,Follow-up Time (Weeks),Diagnostic Criteria
0,"Coric et al., 2010",260,Pexacerfont: 100 mg/day,White,100,24.0,38.5,5,Bristol-Myers Squibb Company,8,DSM-IV-TR
1,"Coric et al., 2010",260,Placebo,White,100,24.7,39.3,3,Bristol-Myers Squibb Company,8,DSM-IV-TR
2,"Coric et al., 2010",260,Escitalopram: 20 mg/day,White,100,23.5,38.8,10,Bristol-Myers Squibb Company,8,DSM-IV-TR


In [184]:
df_truth[df_truth["References"] == "Coric et al., 2010"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
72,"Coric et al., 2010",157,escitalopram: 20 mg/day,White,100.0,23.5,38.8,20.8,Bristol-Myers,8.0,dsm-iv
73,"Coric et al., 2010",157,placebo,White,100.0,24.7,39.3,23.1,Bristol-Myers,8.0,dsm-iv


# Important Notice: ChatGPT has identified a treatment that was omitted in the orignal study for unknown reason: Pexacerfont. I will update df_truth by manually reading the Coric 2010 study and reload it here:

In [185]:
from openpyxl import load_workbook

file_path = '../Downloads/all_studies_final.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active

# Convert to DataFrame
data = sheet.values
columns = next(data)
df_truth = pd.DataFrame(data, columns=columns)

In [186]:
df_truth[df_truth["References"] == "Coric et al., 2010"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
72,"Coric et al., 2010",260,Pexacerfont: 100 mg/day,White,100,24,38.5,26.2,Bristol-Myers,8.0,DSM-IV
73,"Coric et al., 2010",260,Escitalopram: 20 mg/day,White,100,23.5,38.8,20.8,Bristol-Myers,8.0,DSM-IV
74,"Coric et al., 2010",260,Placebo,White,100,24.7,39.3,23.1,Bristol-Myers,8.0,DSM-IV


In [187]:
coric_2010_df = rename_columns(coric_2010_df)
coric_2010_df = preprocess_interventions_and_diag(coric_2010_df)
coric_2010_df = preprocess_numerical_columns(coric_2010_df, numerical_cols)

In [188]:
coric_2010_df

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
0,"Coric et al., 2010",260,pexacerfont: 100 mg/day,White,100,24.0,38.5,5,Bristol-Myers Squibb Company,8,dsm-iv-tr
1,"Coric et al., 2010",260,placebo,White,100,24.7,39.3,3,Bristol-Myers Squibb Company,8,dsm-iv-tr
2,"Coric et al., 2010",260,escitalopram: 20 mg/day,White,100,23.5,38.8,10,Bristol-Myers Squibb Company,8,dsm-iv-tr


In [189]:
coric2010_results = testing(df_truth, coric_2010_df)